# Week 2: Lab 2 - Streaming Ingestion

After completing this lab, you will interact and understand how the streaming ingestion process in performed. This lab is divided into two parts:
1. You will generate data manually and push it to a data stream; after that you will consume the manually generated data from that stream.
2. You will ingest data from a data stream that is fed by a producer. You will consume that data, perform some small transformations and put the data into another data stream. From that data stream, data will be taken by a Kinesis firehose which will store it into an S3 bucket.

Let's start with the lab.

<a name='1'></a>
## 1 - Understanding Streaming Ingestion Process

1.1. Either in the AWS Cloud9 or Jupyter Notebook environment you can find a folder named `src/cli`. There are two Python scripts inside:
- `consumer_from_cli.py`: consumer code that can be used from the CLI to consume from a Kinesis data stream,
- `producer_from_cli.py`: producer code that can be used from the CLI to manually send records to a Kinesis data stream.

1.2. Before you send any information, you have to create a data stream in Kinesis. Go to the AWS Console, search for **Kinesis**, and click on **Create Data Stream**. Name it as `de-c2w2lab2-kinesis-data-stream-cli` and leave the rest as default. Click on **Create data stream** button. Once it is in `Active` Status you can continue with the next steps.

1.3. Either in AWS Cloud9 or Jupyter Notebook terminal activate the `jupyterlab-venv` environment, which is the one that contains the required packages you will use, in particular `boto3`:

```bash
source jupyterlab-venv/bin/activate
```

1.4. Go to the `src/cli/` folder and start consuming from the data stream that you created. The provided consumer script requires a `--stream` flag followed by the data stream name:

```bash
cd src/cli/
python consumer_from_cli.py --stream de-c2w2lab2-kinesis-data-stream-cli
```

This consumer will print the results in the terminal but currently, even if you wait 1 or 2 minutes, nothing should appear as we are consuming from a data stream that currently has no data on it. Let's insert some data manually.

1.5. Create a second terminal, activate the `jupyterlab-venv`, go to the `src/cli/` folder and start producing data:

```bash
source jupyterlab-venv/bin/activate
cd src/cli/
python producer_from_cli.py --stream de-c2w2lab2-kinesis-data-stream-cli --json_string '{"session_id": "a1", "customer_number": 100, "city": "Washington", "country": "USA", "credit_limit": 1000, "browse_history": [ {"product_code": "Product1", "quantity": 2, "in_shopping_cart": true}, {"product_code": "Product2", "quantity": 1, "in_shopping_cart": false}]}'
```

Again, the provided script requires the `--stream` flag with the kinesis data stream where you will send the data and a `--json_string` flag indicating, as a string, the JSON object to be inserted.

Now, if you go to the first terminal (the one executing the consumer), you should see the record that you already sent. Great! You are ingesting streaming data!

1.6. Now, try to insert 4 or 5 more records using the same structure for the `--json_string`. You can see an example of the JSON object at `data/example_record.json` that you can use. Change the payload attributes to make sure you are sending different records. Remember to check in the consumer terminal the produced records.

## Part 2: Streaming EtL process

For the second part, you will be given a producer. You will have to consume data fromm a kinesis data stream and perform a small transformation over the ingested data and put them into another kinesis data streams. Those data streams will be consumed by firehose to store the data in S3 so you can see the result. 


You are going to interact with boto3 to perform not only the consumption from the producer but also to create the necessary resources (Kinesis data streams, firehose and bucket). The first part is to set up the infrastructure, but in order to create the necessary resources, we need to understand the businses requirements. The data that you are going to work with corresponds to the browse history of some users into an e-commerce web page. This is the same data you worked with in C1W4, that has the same structure as the data you produced manually in the first part of this lab: 

```json
{
    "session_id": "a1",
    "customer_number": 100,
    "city": "Washington",
    "country": "USA",
    "credit_limit": 1000,
    "browse_history": [
        {
            "product_code": "Product1",
            "quantity": 2,
            "in_shopping_cart": true
        },
        {
            "product_code": "Product2",
            "quantity": 1,
            "in_shopping_cart": false
        }
    ]
}
```

You can see that the payload contains information about the location of the user. As the e-commerce is based in USA, you are asked to filter the customers by their country, if they are in the USA, orders can be dispatched using a national logistics company, while customers in other countires different from USA will require an international delivery and are dispatched by another department in the company. Given that, you need to send orders by USA customers to a different data stream than those from other countries and each data stream will be sent to a different bucket to be processed in a different way by downstream users. That means that you have to create a data stream, a firehose and a bucket for USA customers and the same set of resources for International orders.

Follow the instructions to create the necessary resources according to the business requirements.

Let's start creating the two necessary buckets. You are given a boto3 function to do it. As a default region, work with `us-east-1` as pointed in the following variable. You can also set the `ACCOUNT_ID` variable with your Vocareum's AWS account ID to be used throughout the lab.

In [1]:
AWS_DEFAULT_REGION = 'us-east-1'
ACCOUNT_ID = '<VOCAREUM_AWS_ACCOUNT_ID>'

In [2]:
import boto3
#boto3.setup_default_session(profile_name='dlai-despec-factored')

First, let's create the buckets, which should follow this convention:
* USA: `de-c2w2lab2-<ACCOUNT-ID>-usa`
* International: `de-c2w2lab2-<ACCOUNT-ID>-international`
  
Change the `<ACCOUNT-ID>` placeholder with your Vocareum's AWS account ID. Then follow these instructions:

1. In the following cell, call the boto3 client with the `'s3'` resource. Set also the region in the client call. Save the client object in the `s3_client` variable.
2. Call the `create_s3_bucket()` function twice to create your two buckets. You can use the `USA_BUCKET` and `INTERNATIONAL_BUCKET` variables to set the bucket names.

In [3]:
def create_s3_bucket(bucket_name: str, region: str) -> None:
    
    s3_client = boto3.client('s3', region_name=region) # Learner
    
    # Create the S3 bucket
    try:
        s3_client.create_bucket(Bucket=bucket_name) # Learner
        print(f"S3 bucket '{bucket_name}' created successfully in region '{region}'.")
    except Exception as e:
        print(f"An error occurred: {e}")


In [4]:
USA_BUCKET = 'de-c2w2lab2-<ACCOUNT-ID>-usa'
INTERNATIONAL_BUCKET = 'de-c2w2lab2-<ACCOUNT-ID>-international'


# Create the USA bucket
create_s3_bucket(bucket_name=USA_BUCKET, region=AWS_DEFAULT_REGION) # Learner
    
# Create the internatonal bucket
create_s3_bucket(bucket_name=INTERNATIONAL_BUCKET, region=AWS_DEFAULT_REGION) # Learner

S3 bucket 'de-c2w2lab2-986402741974-usa' created successfully in region 'us-east-1'.
S3 bucket 'de-c2w2lab2-986402741974-international' created successfully in region 'us-east-1'.


From the Week 1 you know that you can use the aws cli tool to check if the buckets exist. Use the following cell to check if they exist now. Remember that you can also use those commands in the cloud9 terminal.

In [5]:
!aws s3 ls

2023-11-20 02:28:17 aws-athena-query-results-986402741974-us-east-1
2024-01-08 23:18:03 aws-glue-assets-986402741974-us-east-1
2023-11-17 04:04:54 cf-templates-o1db23hlmc8i-us-east-1
2023-12-19 23:48:09 de-c1w2-bucket-test
2024-01-08 23:14:00 de-c1w4-bucket-test
2024-02-07 21:18:40 de-c2w1-s3-lab-bucket
2024-02-15 17:15:09 de-c2w2lab2-986402741974-international
2024-02-15 17:15:09 de-c2w2lab2-986402741974-usa
2024-01-09 22:35:40 sagemaker-studio-986402741974-umhuzu3da8
2024-01-09 22:35:42 sagemaker-us-east-1-986402741974


Now, let's create the Kinesis data streams.

1. Call the boto3 client with the `kinesis` resource.  Store the object in `client`.
2. Use the `create_stream()` method from the client and pass the data stream name and the shard count.
3. Call the `create_kinesis_data_stream()` to create the USA and International data streams. Both of them shoud have a shard count of 2 and should be named with the following convention:
   * USA: `de-c2w2lab2-usa-data-stream`
   * International: `de-c2w2lab2-international-data-stream`
4. With boto3 you can also check the status of the resources, use the provided function `is_stream_ready()` to see if your data streams are ready to be used.

In [6]:
def create_kinesis_data_stream(stream_name: str, shard_count: int = 2) -> None:
    client = boto3.client("kinesis") # Learner
    response = client.create_stream(StreamName=stream_name, ShardCount=shard_count) # Learner
    print("Kinesis data stream created:", response)


def is_stream_ready(stream_name: str) -> None:
    client = boto3.client("kinesis")
    response = client.describe_stream(StreamName=stream_name)
    return response["StreamDescription"]["StreamStatus"] == "ACTIVE"


In [7]:
# Create the USA data stream
create_kinesis_data_stream(stream_name='de-c2w2lab2-usa-data-stream', shard_count=2) # Learner

# Create the International data stream
create_kinesis_data_stream(stream_name='de-c2w2lab2-international-data-stream', shard_count=2) # Learner

Kinesis data stream created: {'ResponseMetadata': {'RequestId': 'f96d5f64-c338-d755-a725-1788c23b8271', 'HTTPStatusCode': 200, 'HTTPHeaders': {'x-amzn-requestid': 'f96d5f64-c338-d755-a725-1788c23b8271', 'x-amz-id-2': 'FfV5qQSpI0lMdGUyqyZJvhY0HLu0EaTSnjMLf7/sx4WEaKySlKDu2RlCUSC9N5YMQYkgVWPmc4uhOz3gQIh+64lXkcKdauy7', 'date': 'Thu, 15 Feb 2024 17:15:24 GMT', 'content-type': 'application/x-amz-json-1.1', 'content-length': '0', 'connection': 'keep-alive'}, 'RetryAttempts': 0}}
Kinesis data stream created: {'ResponseMetadata': {'RequestId': 'f514ae39-b08b-1757-ab5c-e6d5d663e412', 'HTTPStatusCode': 200, 'HTTPHeaders': {'x-amzn-requestid': 'f514ae39-b08b-1757-ab5c-e6d5d663e412', 'x-amz-id-2': 'x2n/zwJ59whnkvXAWVAevk1NTHWhbox07Q8sbQrCcU/t4bUBMpLMxUY5kBvYIZUtmR0BSzCayNCcDlvXguDtzdeBGahAh1EU', 'date': 'Thu, 15 Feb 2024 17:15:24 GMT', 'content-type': 'application/x-amz-json-1.1', 'content-length': '0', 'connection': 'keep-alive'}, 'RetryAttempts': 0}}


In [10]:
# Check if the streams are ready
is_stream_ready(stream_name='de-c2w2lab2-usa-data-stream')

True

After the creation of the data streams and the buckets, you will create the Kinesis Firehose Delivery streams for the two regions, `USA` and `International`. Apart from the bucket and datastreams that you created, you are already provided with log groups, log streams and a role with the necessary permissions to be attached to your delivery stream.
* Firehose IAM role: `de-c2w2lab2-firehose-role`. This role is the only one you have to use when configuring the delivery stream.
* CloudWatch Log Groups: `de-c2w2lab2-firehose-<LOCATION>-log-group`.
* CloudWatch Log Streams: `de-c2w2lab2-<LOCATION>-firehose-log-stream`.

Change the `<LOCATION>` placeholder with `usa` or `international` when using them in the `create_kinesis_firehose()` function.

Follow the instructions to finish the `create_kinesis_firehose()` function below.

1. Call the boto3 client with the `firehose` resource. Assign it to the `client` variable.
2. Use the `create_delivery_stream()` method of the `client` object.


   2.1. Pass it the firehose name.
   
   2.2. Set an appropriate value for the delivery stream type

   2.3. As you will use S3 as destination, search in the [boto3 documentation](https://boto3.amazonaws.com/v1/documentation/api/latest/reference/services/firehose/client/create_delivery_stream.html) for that configuration. You are provided with some parts of the configuration but you will have set role and other resource ARNs based on the resource name, the `account_id` and the `region`. Also, enable the logging options pointing to the corresponding log group and log stream.

3. Finally, configure the Kinesis stream source.

In [3]:
def create_kinesis_firehose( firehose_name: str, stream_name: str, bucket_name: str, role_name: str, log_group: str, log_stream:str, account_id: int, region: str):
    
    client = boto3.client("firehose") # Learner
    response = client.create_delivery_stream(
        DeliveryStreamName=firehose_name, # Learner
        DeliveryStreamType='KinesisStreamAsSource', # Learner
        S3DestinationConfiguration={
            "RoleARN": f"arn:aws:iam::{account_id}:role/{role_name}",
            "BucketARN": f"arn:aws:s3:::{bucket_name}",  # Learner
            "Prefix": "firehose/", # Learner
            "ErrorOutputPrefix": "errors/", # Learner
            "BufferingHints": {"SizeInMBs": 1, "IntervalInSeconds": 60},
            "CompressionFormat": "UNCOMPRESSED",  
            "CloudWatchLoggingOptions": {
                "Enabled": True,
                "LogGroupName": log_group, 
                "LogStreamName": log_stream
            }, # Learner
            "EncryptionConfiguration": {"NoEncryptionConfig": "NoEncryption"},
        },
        KinesisStreamSourceConfiguration={
            "KinesisStreamARN": f"arn:aws:kinesis:{region}:{account_id}:stream/{stream_name}", # Learner
            "RoleARN": f"arn:aws:iam::{account_id}:role/{role_name}", # Learner
        },
    )
    
    print("Kinesis Firehose created:", response)


Now, create the two delivery streams, for USA and for International orders.

In [4]:
create_kinesis_firehose(firehose_name='de-c2w2lab2-firehose-usa', 
                        stream_name='de-c2w2lab2-usa-data-stream', 
                        bucket_name='de-c2w2lab2-106480627864-usa', 
                        role_name='de-c2w2lab2-firehose-role', 
                        log_group='de-c2w2lab2-firehose-usa-log-group',
                        log_stream='de-c2w2lab2-usa-firehose-log-stream',
                        account_id=ACCOUNT_ID,
                        region=AWS_DEFAULT_REGION
                       )

Kinesis Firehose created: {'DeliveryStreamARN': 'arn:aws:firehose:us-east-1:986402741974:deliverystream/de-c2w2lab2-firehose-usa', 'ResponseMetadata': {'RequestId': 'fed403fc-3b92-429e-a09c-4ffa383b395c', 'HTTPStatusCode': 200, 'HTTPHeaders': {'x-amzn-requestid': 'fed403fc-3b92-429e-a09c-4ffa383b395c', 'x-amz-id-2': 'YQsbJGtpw3rZVmTOkDuWHbh0fjUI1y0LrmlmNFTs/+eg10eyeB7WYS6JHg1HzYfqovwWpzd6OUz4hEgAnnX8gU9RkAzExoyv', 'content-type': 'application/x-amz-json-1.1', 'content-length': '103', 'date': 'Thu, 15 Feb 2024 17:28:37 GMT'}, 'RetryAttempts': 0}}


In [5]:
create_kinesis_firehose(firehose_name='de-c2w2lab2-firehose-international', 
                        stream_name='de-c2w2lab2-international-data-stream', 
                        bucket_name='de-c2w2lab2-106480627864-international', 
                        role_name='de-c2w2lab2-firehose-role', 
                        log_group='de-c2w2lab2-firehose-international-log-group',
                        log_stream='de-c2w2lab2-international-firehose-log-stream',
                        account_id=ACCOUNT_ID,
                        region=AWS_DEFAULT_REGION
                       )

Kinesis Firehose created: {'DeliveryStreamARN': 'arn:aws:firehose:us-east-1:986402741974:deliverystream/de-c2w2lab2-firehose-international', 'ResponseMetadata': {'RequestId': 'e21ae683-cb68-85d9-bc52-aaa865f6dacb', 'HTTPStatusCode': 200, 'HTTPHeaders': {'x-amzn-requestid': 'e21ae683-cb68-85d9-bc52-aaa865f6dacb', 'x-amz-id-2': 'pnlttJ7Ce3mqdH/KAeT304XhJ/0CUrtsGf0HnfxK5yO5zrm4GdIAApa4BOof6hDcqESrqYSMVuqf1fGXJRgk0YmtWcF0Mo1t', 'content-type': 'application/x-amz-json-1.1', 'content-length': '113', 'date': 'Thu, 15 Feb 2024 17:29:15 GMT'}, 'RetryAttempts': 0}}


Now that your resources are created, it is time to consume the data. For a better learning experience, the producer generates data dynamically with an average mean time between records of 10 seconds, so take that into account when you are consuming your data and want to visualize it. During this consumption, you will perform some small transformations over the records before sending them to the new data streams created with boto3. Follow the instructions below:

1. Go to the `streaming/consumer_EtL/src` folder. Open the script called `consumer.py`. This file is similar to the one you used for the first part of the lab but has some additions.
   
2. Search for the `poll_shards()` function. This is the place where data is read from the stream and also where you will add some small transformations to each record. 
   
3. Search for the loop `for record in records_response["Records"]`. The next line `order = json.loads(record["Data"].decode("utf-8"))` loads the data and converts the record into a dictionary object `order` that you can modify.

4. First transformation is adding the processing timestamp. Add a new key to the `order` dictionary named `"processing_timestamp"` and as value set the current timestamp. Use the `datetime` package for that purpose.

5. You are also asked to add 3 metrics, the total products in the shopping cart, the overall product quantity and the different products in the browse history. First, before the loop `for product in order["browse_history"]` create two variables `overall_product_quantity` and `overall_in_shopping_cart`. Set them as 0. This loop goes through each of the elements of the browse history list. If you are confused about the structure of the records, you can check it again in the example presented at the beginning of the second part of the lab.
6. Count the overall product quantity in the browse history. For that, you have to access to the `"quantity"` key of each product and add it to the `overall_product_quantity` variable. Make sure to convert the value to integer before adding it. This value arrives as string due to the decoding process performed when converting the record into a python dictionary.
7. Count the number of products in the shopping cart. For that, create a condition over the `"in_shopping_cart"` key of each `product` to accept only those that are in the shopping cart. Then, accumulate the `"quantity"` value in the `overall_in_shopping_cart` varible. 
8. Outside the loop `for product in order["browse_history"]` add the computed values to the `order` dictionary. Use the keys `"overall_product_quantity"` and `"overall_in_shopping_cart"` for that purpose. Also, create a new key `"total_different_products"` where you will compute the different products in the browse history. You can do it by computing the lenght of the order's browse history list.
9. As the last step, you will create a filter to send the order to the USA or the International data stream. In the same `poll_shards()` function you can see that it receives the `kinesis_dest_stream_names` parameter, which is a dictionary with two keys:
    * `"USA"`: The value contains the data stream where USA orders will be placed.
    * `"International"`: The value contains the data stream where International orders will be placed.
    
    In that function, just below where you were creating your metrics you  will find a call to the `kinesis.put_record()` method.  In the `StreamName`, create a filter where evaluates the `"country"` key of the order and depending on the value, calls the `kinesis_dest_stream_names` dictionary to send the corresponding data stream name.

After those changes, in the cloud9 terminal go to the `streaming/consumer_EtL/src` folder and execute the consumer with the following command, making sure to change the placeholders with the corresponding data stream names that you created previously.

```bash
python consumer.py --source_stream de-c2w2lab2-kinesis-data-stream --dest_streams '{"USA": "<YOUR-USA-DATA-STREAM>", "International": "<YOUR-INTERNATIONAL-DATA-STREAM>"}'
```

As a quick check you can also use another cloud9 terminal, activate the `jupyterlab-venv` environment and use the consumer of the first part of the lab located at `streaming/CLI/consumer_from_cli/src/consumer.py` pointing to either the USA or International data stream to  inspect visually your transformed data.

Finally, you can inspect from the AWS Console each of the S3 buckets to see when the data is saved. This process can take around 5-7 minutes to start seeing any file in the S3 bucket after the transformations are sent to the data streams. 